# Week 09: Exercise 1

## Database

- JHU data on covid cases/death in US Counties
- Daily update in September 2020
- Database is normalized
  - tables for county and state
- Other tables exist
  - `demography` 
  - `election_2016`

In [1]:
!wget https://www.dropbox.com/s/wkhjoah228gmhtl/covid.sqlite?dl=1 -O covid.sqlite

--2020-12-01 14:27:29--  https://www.dropbox.com/s/wkhjoah228gmhtl/covid.sqlite?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601d:1::a27d:501
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/wkhjoah228gmhtl/covid.sqlite [following]
--2020-12-01 14:27:29--  https://www.dropbox.com/s/dl/wkhjoah228gmhtl/covid.sqlite
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc3010c16d1e54d42e0081d3a0d.dl.dropboxusercontent.com/cd/0/get/BEMYeCvx9ULFk1N4n-dWrwC5NOrgcbZJ3RnsrwJUnSih8zQDZR2N1074IG03nve0iHKCVb1Hgm5tuEPmBeXKxU9f_30oQphigNSxv1fIVcPELsb5-fstUft5ILbKlLSNjPY/file?dl=1# [following]
--2020-12-01 14:27:29--  https://ucc3010c16d1e54d42e0081d3a0d.dl.dropboxusercontent.com/cd/0/get/BEMYeCvx9ULFk1N4n-dWrwC5NOrgcbZJ3RnsrwJUnSih8zQDZR2N1074IG03nve0iHKCVb1Hgm5tuEPmBeXKxU9f_30oQphigNSxv1fIVcPELsb5

## Import packages

In [2]:
import pandas as pd
import sqlite3

## Create a connection object

In [4]:
con_cov = sqlite3.connect('covid.sqlite')

## Check tables

In [5]:
pd.read_sql_query("SELECT * FROM sqlite_master WHERE type='table'", con_cov)

,type,name,tbl_name,rootpage,sql
0,table,covid,covid,2,"CREATE TABLE ""covid"" (\n""fips"" INTEGER,\n ""da..."
1,table,county,county,940,"CREATE TABLE ""county"" (\n""fips"" INTEGER,\n ""c..."
2,table,state,state,968,"CREATE TABLE ""state"" (\n""state_code"" TEXT,\n ..."
3,table,demography,demography,970,"CREATE TABLE ""demography"" (\n""fips"" INTEGER,\n..."
4,table,election_2016,election_2016,1174,"CREATE TABLE ""election_2016"" (\n""fips"" INTEGER..."


In [ ]:
df_dbinfo

In [6]:
pd.read_sql_query("SELECT * FROM covid LIMIT 5", con_cov)

,fips,date,cases,deaths
0,1001,2020-09-01,1354,23
1,1001,2020-09-02,1345,23
2,1001,2020-09-03,1349,23
3,1001,2020-09-04,1355,23
4,1001,2020-09-05,1371,23


In [7]:
pd.read_sql_query("SELECT * FROM county LIMIT 5", con_cov)

,fips,county,state_code
0,1001,Autauga,AL
1,1003,Baldwin,AL
2,1005,Barbour,AL
3,1007,Bibb,AL
4,1009,Blount,AL


In [8]:
pd.read_sql_query("SELECT * FROM state LIMIT 5", con_cov)

,state_code,state
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


In [10]:
pd.read_sql_query("SELECT * FROM demography LIMIT 5", con_cov)

,fips,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,55036,26899,28137,2.7,75.4,18.9,0.3,0.9,0.0,41016,55317,2838,27824,2024,13.7,20.1,35.3,18.0,23.2,8.1,15.4,86.0,9.6,0.1,0.6,1.3,2.5,25.8,24112,74.1,20.2,5.6,0.1,5.2
1,1003,203360,99527,103833,4.4,83.1,9.5,0.8,0.7,0.0,155376,52562,1348,29364,735,11.8,16.1,35.7,18.2,25.6,9.7,10.8,84.7,7.6,0.1,0.8,1.1,5.6,27.0,89527,80.7,12.9,6.3,0.1,5.5
2,1005,26201,13976,12225,4.2,45.7,47.8,0.2,0.6,0.0,20269,33368,2551,17561,798,27.2,44.9,25.0,16.8,22.6,11.5,24.1,83.4,11.1,0.3,2.2,1.7,1.3,23.4,8878,74.1,19.1,6.5,0.3,12.4
3,1007,22580,12251,10329,2.4,74.6,22.0,0.4,0.0,0.0,17662,43404,3431,20911,1889,15.2,26.6,24.4,17.6,19.7,15.9,22.4,86.4,9.5,0.7,0.3,1.7,1.5,30.0,8171,76.0,17.4,6.3,0.3,8.2
4,1009,57667,28490,29177,9.0,87.4,1.5,0.3,0.1,0.0,42513,47412,2630,22021,850,15.6,25.4,28.5,12.9,23.3,15.8,19.5,86.8,10.2,0.1,0.4,0.4,2.1,35.0,21380,83.9,11.9,4.0,0.1,4.9


In [9]:
pd.read_sql_query("SELECT * FROM election_2016 LIMIT 5", con_cov)

,fips,pct_report,votes,total_votes,pct
0,6037,1.0000,542591.0,2314275.0,0.234454
1,17031,0.9746,440213.0,2055215.0,0.214193
2,48201,1.0000,544960.0,1302887.0,0.418271
3,4013,1.0000,590465.0,1201934.0,0.491262
4,12086,1.0000,333666.0,978670.0,0.340938


### Combine covid and county

In [11]:
pd.read_sql_query('''
SELECT a.*, b.county, b.state_code 
  FROM covid AS a 
  INNER JOIN county AS b 
  ON a.fips = b.fips
''', con_cov)

,fips,date,cases,deaths,county,state_code
0,1001,2020-09-01,1354,23,Autauga,AL
1,1001,2020-09-02,1345,23,Autauga,AL
2,1001,2020-09-03,1349,23,Autauga,AL
3,1001,2020-09-04,1355,23,Autauga,AL
4,1001,2020-09-05,1371,23,Autauga,AL
...,...,...,...,...,...,...
97225,56045,2020-09-26,36,0,Weston,WY
97226,56045,2020-09-27,36,0,Weston,WY
97227,56045,2020-09-28,36,0,Weston,WY
97228,56045,2020-09-29,37,0,Weston,WY


### Combine covid, county, and state

In [14]:
pd.read_sql_query('''
SELECT a.*, b.county, b.state_code, c.state
  FROM covid AS a 
  INNER JOIN county AS b 
  ON a.fips = b.fips
  INNER JOIN state as c 
  ON b.state_code = c.state_code''', con_cov)

,fips,date,cases,deaths,county,state_code,state
0,1001,2020-09-01,1354,23,Autauga,AL,Alabama
1,1001,2020-09-02,1345,23,Autauga,AL,Alabama
2,1001,2020-09-03,1349,23,Autauga,AL,Alabama
3,1001,2020-09-04,1355,23,Autauga,AL,Alabama
4,1001,2020-09-05,1371,23,Autauga,AL,Alabama
...,...,...,...,...,...,...,...
97225,56045,2020-09-26,36,0,Weston,WY,Wyoming
97226,56045,2020-09-27,36,0,Weston,WY,Wyoming
97227,56045,2020-09-28,36,0,Weston,WY,Wyoming
97228,56045,2020-09-29,37,0,Weston,WY,Wyoming


# Replicate the work in Assignment 1 using SQL queries

In this exercise, you will do the same work you have done with SQL queries

## Mergeing with the demography data


### Check the contents of demography table


- Merge covid, county, state, and demography tables
  - select the data on 30 September
- Calculate the deaths/cases per 1000 population in the merged table
- Create a new pandas DataFrame from query
- Dump the content of DataFrame as a table (named `sep30_table` in this database) using `.to_sql`


In [59]:
df_sep30 = pd.read_sql_query('''
SELECT a.*, b.county, b.state_code, c.state, d.*,
  (deaths + 0.0) / a.deaths / TotalPop *1000 AS Deaths_Per_1000
  FROM covid AS a 
  INNER JOIN county AS b 
  ON a.fips = b.fips
  INNER JOIN state as c 
  ON b.state_code = c.state_code
  INNER JOIN demography AS d
  ON a.fips = d.fips
  WHERE date = '2020-09-30'
  ''', con_cov)

In [57]:
df_sep30[['deaths', 'TotalPop', 'Deaths_Per_1000']].head()

,deaths,TotalPop,Deaths_Per_1000
0,27,55036,0.018170
1,52,203360,0.004917
2,7,26201,0.038166
3,11,22580,0.044287
4,15,57667,0.017341


In [64]:
df_sep30.columns

Index(['fips', 'date', 'cases', 'deaths', 'county', 'state_code', 'state',
       'fips', 'TotalPop', 'Men', 'Women', 'Hispanic', 'White', 'Black',
       'Native', 'Asian', 'Pacific', 'VotingAgeCitizen', 'Income', 'IncomeErr',
       'IncomePerCap', 'IncomePerCapErr', 'Poverty', 'ChildPoverty',
       'Professional', 'Service', 'Office', 'Construction', 'Production',
       'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp', 'WorkAtHome',
       'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork', 'SelfEmployed',
       'FamilyWork', 'Unemployment', 'Deaths_Per_1000'],
      dtype='object')

In [63]:
df_sep30 = df_sep30.loc[:, ~df_sep30.duplicated()]pd.read

IndexingError: ignored

In [61]:
df_sep30.to_sql('sep30_table', con_cov, if_exists = 'replace')

OperationalError: ignored


- Find the maximum of highest deaths/cases per 1000 population and their county names 



In [67]:
pd.read_sql_query("SELECT * FROM sep_30_table ORDER BY deaths_per_1000 DESC LIMIT1", con_cov)

DatabaseError: ignored

- Aggregate the data in `sep30_table` at the state level and find the following
  - Calculate the number of total cases/deaths per 1000 population
  - Which state had the highest cases/deaths, how many cases/deaths?


In [71]:
pd.read_sql_query('''
"SELECT state_code, state, 
  SUM(deaths) AS total_deaths,
  SUM(cases) AS total_cases,
  (SUM(deaths) + 0.0) / SUM(TotalPop) AS deaths_per_1000
  (SUM(cases) + 0.0) / SUM(TotalPop) AS cases_per_1000
  FROM sep30_table 
  GROUP BY state
  ORDER BY total_deaths DESC
  ''', con_cov)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 2))



DatabaseError: ignored

## Merge the presidential election data and explore the relations


### Check the contents of election_2016 table

In this table, pct is the Trump vote percentage

- Check the table content


- Merge the presidential election results with `sep30_table`
- Explore the relations between Trump vote percentage and Covid-19 variables. 

In [73]:
pd.read_sql_query('''
SELECT s.*, e.pct as TRUMP-pct
  FROM sep30_table as s
  INNER JOIN election_2016 as e
  ON s.fips = e.fips
''', con_cov).loc[:, ['trump_pct', 'White', 'Black', 'Hispanic', 'deaths_per_1000', 'cases_per_1000']].corr()

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



DatabaseError: ignored